# Demo for LumiSpy package working with AttoLight SEM CL data

### This notebook needs to be updated, as loading of Attolight data should not be done using the `.sur` file format and the `.bin` file will not be loaded.

This notebook shows:

- Loading a `HYPCard.bin` file correctly with the lumispy load function
- Plotting cathodoluminescence data in different ways
- Dealing with metadata
- Correcting for spectral defects

Import packages:

In [ ]:
%matplotlib notebook 
# Use '%matplotlib widget' in JupterLab and '%matplotlib notebook' in JupyterNotebook for interactive inline functionality (e.g. on binder)
#For pop-up window plots on your local computer, use '%matplotlib tk' or '%matplotlib qt' instead
import hyperspy.api as hs
import lumispy as lum

## Loading HYPCard files

Use the `load_hypcard` function to load the AttoLight SEM specific files. State the relative path to the `HYPCard.bin` file.

*You can also leave the path empty. A pop-up window will appear to select the `HYPCard.bin` file from the browser.*

In [ ]:
path = "demo-files/load_from_AttoLightSEM/HYPCard.bin"
cl_sem = lum.load(path)
cl_sem

In [ ]:
cl_sem.plot()

## Correction of spectral defects

Let's start some post-processing methods.

### Background removal

Check if a background was saved during image acquisiton (in the AttoLight system). If so, it has been stored automatically in the signal object.

Otherwise, manually load a background file, using `np.loadtxt(path_to_file)`.

In [ ]:
cl_sem.metadata.Signal.background is not None

Run background subtraction. No arguments required if the background is stored in the signal object. Otherwise, a background array of shape `[wavelength, bacgkround intensity]` needs to be passed as a `background` argument.

Optional arguments are `background`: to subtract an external background file, `inplace`: if `True` it will overwrite the original signal, if `False` if outputs a copy of the signal, with the background substracted (this is by default).

In [ ]:
cl_sem_bkg = cl_sem
cl_sem_bkg = cl_sem.background_subtraction()

### Correction of acquisition defects

Correct for the intrinsic shift caused by the misalignment of the grating with the spectrometer aperture centre. For the AttoLight in Cambridge, we have created a function to correct for it

In [ ]:
cl_sem_bkg.correct_grating_shift()

The edges of the scan get higher intensities, so they can be cropped. 
If you set the `inplace` parameter to `True` the original CLSEMSpectrum object will be modified, if `False` a cropped copy of it will be created.

In [ ]:
cl_sem_bkg = cl_sem_bkg.crop_edges(crop_px=5)

There are also corrections for the cosmic rays (pixels with sharp spikes):

In [ ]:
cl_sem_bkg_spikes = cl_sem_bkg.cosmic_rays_subtraction(extra_percent=50)

## Plotting data

Plot the corrected data:

In [ ]:
cl_sem_bkg_spikes.plot()

### Panchromatic image:

In [ ]:
cl_sem_bkg_spikes.T.mean().plot(cmap='viridis')

Plot the average CL spectrum:

In [ ]:
cl_sem_bkg_spikes.mean().plot()

### CL signal with the SEM image as navigator axis

Plot with the SEM image instead of the panchromatic navigation image.

*Note: if you cropped the edges before, you'll also need to crop the SEM image*

In [ ]:
#Find the folder, path of the SE_Scan file and load the file
import os
se_path = os.path.join(os.path.split(path)[0], "SE_Scan*.png")
SE = lum.hyperspyload(se_path)

In [ ]:
#Crop the SEM image (if you have cropped the edges beforehand) and plot
try:
    crop = cl_sem_bkg_spikes.metadata.Signal.cropped_edges
    size_x = cl_sem_bkg_spikes.metadata.Acquisition_instrument.SEM.resolution_x
    size_y = cl_sem_bkg_spikes.metadata.Acquisition_instrument.SEM.resolution_x
    SE = SE.isig[crop:size_x-crop, crop:size_y-crop]
    cl_sem_bkg_spikes.plot(navigator=SE)
#Simply plot with the SEM image
except:
    cl_sem_bkg_spikes.plot(navigator=SE)

## Metadata

All the microscope & spectrometer parameters are saved in the metadata, as well as the data processing steps applied. It can be assessed via:

In [ ]:
cl_sem_bkg_spikes.metadata